# REST API Mutiple Image Samples

## Objective
Managing multiple image inputs in GPT-4V.

## Time

You should expect to spend 5-10 minutes running this sample.

## Before you begin

#### Installation

In [ ]:
%pip install -r ../requirements.txt

### Parameters
You need to set a series of configurations such as GPT-4V_DEPLOYMENT_NAME, OPENAI_API_BASE, OPENAI_API_VERSION.

Add "OPENAI_API_KEY" as variable name and \<Your API Key Value\> as variable value in the environment variables.
 <br>
      
      WINDOWS Users: 
         setx OPENAI_API_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"

      MACOS/LINUX Users: 
         export OPENAI_API_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"

In [ ]:
# Setting up the deployment name
deployment_name: str = "<your GPT-4V deployment name>"
# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base: str = "<your resource base URL>"
# Currently OPENAI API have the following versions available: 2022-12-01.
# All versions follow the YYYY-MM-DD date structure.
openai_api_version: str = "<your OpenAI API version>"

should_cleanup: bool = False

## Connect to your project
To start with let us create a config file with your project details. This file can be used in this sample or other samples to connect to your workspace.

In [ ]:
import json
from pathlib import Path

config = {
    "GPT-4V_DEPLOYMENT_NAME": deployment_name,
    "OPENAI_API_BASE": openai_api_base,
    "OPENAI_API_VERSION": openai_api_version,
}

p = Path("../config.json")

with p.open(mode="w") as file:
    file.write(json.dumps(config))

## Run this Example

In [ ]:
import base64
from IPython.display import Image, display
from pathlib import Path
import sys

parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from shared_functions import call_GPT4V_image

# Defect Detector
reference_image_file_path = "DefectDetectorReference.jpg"
with Path(reference_image_file_path).open("rb") as image_file:
    encoded_reference_image = base64.b64encode(image_file.read()).decode("utf-8")

test_image_file_path = "DefectDetectorTest.jpg"
with Path(test_image_file_path).open("rb") as image_file:
    encoded_test_image = base64.b64encode(image_file.read()).decode("utf-8")

sys_message = """
You're a professional defect detector.
Your job is to compare the test image with reference image, please answer the question with "No defect detected" or "Defect detected", 
also explain your decision as detail as possible."
"""

messages = [
    {"role": "system", "content": [{"type": "text", "text": sys_message}]},
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Here is the reference image",  # Pass the prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_reference_image}"  # Pass the encoded reference image
                },
            },
            {
                "type": "text",
                "text": "Here is the test image",  # Pass the prompt
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{encoded_test_image}"},  # Pass the encoded test image
            },
        ],
    },
]

try:
    response_content = call_GPT4V_image(messages)
    print("Reference image:")
    display(Image(reference_image_file_path))
    print("Test image:")
    display(Image(test_image_file_path))
    print(response_content["choices"][0]["message"]["content"])  # Print the content of the response
except Exception as e:
    print(f"Failed to call GPT-4V API. Error: {e}")

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group).

In [ ]:
if should_cleanup:
    # {{TODO: Add resource cleanup}}
    pass